# UDA-Hub: Multi-Agent Customer Support for CultPass

This notebook demonstrates the UDA-Hub multi-agent system using LangGraph's Supervisor pattern.

**Prerequisites:** Run `01_external_db_setup.ipynb` and `02_core_db_setup.ipynb` first.

## Setup

In [ ]:
import logging
import json
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage

load_dotenv()

# Configure structured JSON logging to see agent decisions
from agentic.logging_config import setup_logging
setup_logging(level=logging.INFO)

logger = logging.getLogger('uda-hub-demo')

In [ ]:
from agentic.workflow import orchestrator
print('\u2705 Orchestrator loaded successfully')

In [ ]:
def run_scenario(message: str, thread_id: str = "demo"):
    """Run a single message through the orchestrator and display the response."""
    print(f"\n{'='*60}")
    print(f"Customer: {message}")
    print(f"{'='*60}")
    
    result = orchestrator.invoke(
        {"messages": [HumanMessage(content=message)]},
        config={"configurable": {"thread_id": thread_id}},
    )
    
    response = result["messages"][-1].content
    print(f"\nAssistant: {response}")
    print(f"{'='*60}")
    return result

## End-to-End Scenarios

### Scenario 1: General Question - KB Article Resolution

In [ ]:
result1 = run_scenario(
    "How do I reserve a spot for a cultural experience?",
    thread_id="scenario-1"
)

### Scenario 2: Account Inquiry - Email Lookup

In [ ]:
result2 = run_scenario(
    "Can you look up my account? My email is bob.stone@granite.com. What's my subscription status?",
    thread_id="scenario-2"
)

### Scenario 3: Action - Cancel Reservation Flow

In [ ]:
result3 = run_scenario(
    "I need to cancel my reservation. My email is alice.kingsley@wonderland.com. "
    "Can you show me my reservations and cancel the first one?",
    thread_id="scenario-3"
)

### Scenario 4: Escalation - Low RAG Confidence

In [ ]:
result4 = run_scenario(
    "I want to file a formal complaint about discrimination I experienced at one of your partner venues. "
    "I need to speak to a manager or legal representative immediately.",
    thread_id="scenario-4"
)

### Scenario 5: Multi-Turn Conversation

In [ ]:
# Turn 1
result5a = run_scenario(
    "Hi, I'm having trouble with my subscription.",
    thread_id="scenario-5"
)

In [ ]:
# Turn 2 - continues in same thread
result5b = run_scenario(
    "My email is eva.green@ecosoul.net. Can you check my subscription status?",
    thread_id="scenario-5"
)

In [ ]:
# Turn 3 - follow-up action
result5c = run_scenario(
    "I'd like to pause my subscription for now.",
    thread_id="scenario-5"
)

## Memory Demonstration

### Short-term Memory (MemorySaver)

In [ ]:
# Inspect the state history for scenario-5 to see conversation continuity
history = list(orchestrator.get_state_history(
    config={"configurable": {"thread_id": "scenario-5"}}
))

print(f"State history entries for scenario-5: {len(history)}")
print(f"\nMessages in latest state:")
for msg in history[0].values["messages"]:
    role = msg.__class__.__name__.replace('Message', '')
    content = msg.content[:100] + '...' if len(msg.content) > 100 else msg.content
    if content:  # Skip empty messages
        print(f"  [{role}] {content}")

### Long-term Memory (Database Persistence)

In [ ]:
from agentic.memory.persistence import (
    save_message, load_conversation_history,
    save_resolution, load_resolutions_for_user,
    save_customer_preference, load_customer_preferences,
)
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from data.models.udahub import Ticket
from agentic.config import UDAHUB_DB_PATH

# Get existing ticket ID
engine = create_engine(f"sqlite:///{UDAHUB_DB_PATH}", echo=False)
Session = sessionmaker(bind=engine)
session = Session()
ticket = session.query(Ticket).first()
session.close()

if ticket:
    # --- Message Persistence ---
    save_message(ticket.ticket_id, "ai", "I found your account. Let me help you with your login issue.")
    history = load_conversation_history(ticket.ticket_id)
    print("=== Conversation History ===")
    for msg in history:
        print(f"  [{msg['role']}] {msg['content'][:80]}")

    # --- Resolution Tracking (cross-session learning) ---
    save_resolution(
        ticket_id=ticket.ticket_id,
        summary="Resolved login issue by providing password reset instructions from KB",
        agent_name="knowledge_agent",
        resolution_type="kb_article",
        articles_used=["login-issues-article"],
        tools_used=["search_knowledge"],
    )
    print("\n=== Past Resolutions for User ===")
    resolutions = load_resolutions_for_user("a4ab87")
    for r in resolutions:
        print(f"  [{r['resolution_type']}] {r['summary'][:80]}")
        print(f"    Agent: {r['agent']}, Tools: {r['tools_used']}")

    # --- Customer Preferences (cross-session personalization) ---
    save_customer_preference("a4ab87", "language", "pt-BR")
    save_customer_preference("a4ab87", "contact_method", "chat")
    save_customer_preference("a4ab87", "notification_pref", "email")

    print("\n=== Customer Preferences ===")
    prefs = load_customer_preferences("a4ab87")
    for key, value in prefs.items():
        print(f"  {key}: {value}")
else:
    print("No tickets found. Run notebook 02 first.")

## Interactive Chat

Run the cell below to start an interactive chat session. Type 'q' to quit.

In [ ]:
# Interactive chat — only works when running the notebook manually in Jupyter.
# Uncomment the lines below to start a chat session. Type 'q' to quit.

# from utils import chat_interface
# chat_interface(orchestrator, "interactive-1")